<a href="https://colab.research.google.com/github/Karthick47v2/mcq-generator/blob/main/summarize_n_extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# TODO : OPTIMIZEEEEEE

# https://turbolab.in/abstractive-summarization-using-googles-t5/

#OPTIMIZE
# https://www.youtube.com/watch?v=2glb3BfOSVE

In [ ]:
# DONT FORGET TO CHANGE TO CUDA VERSION
# !pip3 install torch==1.8.2+cpu torchvision==0.9.2+cpu torchaudio==0.8.2 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html
!pip3 install transformers
!pip3 install SentencePiece
!pip3 install git+https://github.com/boudinfl/pke.git

In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import pke
import string

In [3]:
T5_MODEL_PATH = 't5-large'

model = T5ForConditionalGeneration.from_pretrained(T5_MODEL_PATH)
tokenizer = T5Tokenizer.from_pretrained(T5_MODEL_PATH)

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.75G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
def summarize(text):
  # preprocess
  text = text.strip().replace("\n","")
  # https://huggingface.co/docs/transformers/preprocessing  - Tokenize
  input_tokens = tokenizer.encode("summarize: " + text, return_tensors='pt', max_length= 1000, padding='max_length', truncation=True)

  summary_encoded = model.generate(input_tokens, 
                                 num_beams=3,
                                 no_repeat_ngram_size=3, 
                                 length_penalty=2.0, 
                                 min_length=50, 
                                 max_length=500, 
                                 early_stopping=True)
  # decode summarized token
  output = tokenizer.decode(summary_encoded[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
  return output

In [76]:
def extract_keywords(text, kw_pop):
  ex = pke.unsupervised.MultipartiteRank()
  ex.load_document(text)   

  pos = {'PROPN','NOUN'}
  stoplist = list(string.punctuation) 
  stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
  stoplist += stopwords.words('english')
  ex.candidate_selection(pos=pos, stoplist=stoplist)

  ex.candidate_weighting(alpha=1.1, threshold=0.75, method='average')
  return ex.get_n_best(n=kw_pop)

In [77]:
def filter_keywords(original, summarized, kw_pop=15):
  orig_kw = extract_keywords(original, kw_pop)
  sum_kw = extract_keywords(summarized, kw_pop)

  orig_ls = []
  sum_ls = []

  for i in orig_kw:
    orig_ls.append(i[0])

  for i in sum_kw:
    sum_ls.append(i[0])

  orig_ls = set(orig_ls)
  return list(orig_ls.intersection(sum_ls))

In [73]:
text = '''

The US has "passed the peak" on new coronavirus cases, President Donald Trump said and predicted that some states would reopen this month. The US has over 637,000 confirmed Covid-19 cases and over 30,826 deaths, the highest for any country in the world. At the daily White House coronavirus briefing on Wednesday, Trump said new guidelines to reopen the country would be announced on Thursday after he speaks to governors. "We'll be the comeback kids, all of us," he said. "We want to get our country back." The Trump administration has previously fixed May 1 as a possible date to reopen the world's largest economy, but the president said some states may be able to return to normalcy earlier than that.

'''

summary = summarize(text)
print(summary)

the us has over 637,000 confirmed covid-19 cases and over 30,826 deaths. "we'll be the comeback kids, all of us," says president Donald Trump. some states may be able to return to normality earlier than may 1, he says.


In [78]:
filtered_kw = filter_keywords(text, summary)